Calculation of odds ratios (including code for Fig. 4A)

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
import random
import glob
import pyreadr
from operator import itemgetter
import itertools
from itertools import groupby
import seaborn as sns
import matplotlib.ticker as ticker
import sys
import scipy.stats  as stats
pd.set_option('display.max_columns', None)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
import statsmodels.api as sm

In [ ]:
tumor_roi_dat = pd.read_csv('new_annotation_Feb2021_KW.txt', delimiter = "\t")
protein_dat=pd.read_csv('ERCC_IgG_norm.csv')
nuc_count = pd.read_csv('nuc_count.csv')

protein_dat=protein_dat.rename({'Unnamed: 0': 'Protein'}, axis=1) 

all_rois=list(protein_dat.columns)[1:]

tumor_core_roi_dict={}
excluded_rois=[]
for loc in range(len(tumor_roi_dat)):
    cur_row = tumor_roi_dat.iloc[loc]
    roi = cur_row['ROI']
    roi_num=roi.split('_')[1]
    if len(roi_num)==1:
        roi = 'ROI_0'+roi_num
    #print(cur_row)
    if roi in all_rois:
        key = (cur_row['Tumor_ID'],cur_row['Core_ID'])
        if key[0]==21 or key[0]==8:
            excluded_rois.append(roi)
        else:
            if key in tumor_core_roi_dict:
                tumor_core_roi_dict[key].append(roi)
            else:
                tumor_core_roi_dict[key] = [roi]            
            

tumor_ids=list(tumor_roi_dat['Tumor_ID'])
core_ids=list(tumor_roi_dat['Core_ID'])
fish_cores=list(tumor_roi_dat['FISH_core'])

tumor_core_to_shreya_base={(tumor_ids[i],core_ids[i]):fish_cores[i] for i in range(len(tumor_roi_dat))}
tumor_core_to_shreya={}
for k,v in tumor_core_to_shreya_base.items():
    if k in tumor_core_roi_dict.keys():
        tumor_core_to_shreya[k]=v
all_proteins_cores_shreya=list(tumor_core_to_shreya.values())

tumor_roi_dict={}
for k, rois in tumor_core_roi_dict.items():
    tumor=k[0]
    if tumor in tumor_roi_dict.keys():
        tumor_roi_dict[tumor]=tumor_roi_dict[tumor]+rois
    else:
        tumor_roi_dict[tumor]=rois
            
nuc_count=nuc_count.rename({'Unnamed: 0': 'ROI'}, axis=1) 

tumor_protein_dfs=[]
for tumor,rois in tumor_roi_dict.items():
    cur_df=protein_dat[rois]
    cur_df.index=protein_dat['Protein']
    tot_nuc_count=nuc_count[nuc_count['ROI'].isin(protein_dat[rois])]['nuclei_count'].sum()
    for roi in rois:
        cur_weight=list(nuc_count[nuc_count['ROI']==roi]['nuclei_count'])[0]/tot_nuc_count
        cur_df[roi]=cur_df[roi]*cur_weight 
    cur_df=cur_df.sum(axis=1).to_frame().rename(columns={0:tumor})
    tumor_protein_dfs.append(cur_df)
protein_dat_by_tumor=pd.concat(tumor_protein_dfs,1)

protein_dat_by_tumor=protein_dat_by_tumor.T

In [ ]:
filenames = glob.glob("FISH_data/table*")
dfs_o = {}

for filename in filenames:
    df_o = pd.read_csv(filename)
    shreya_core_num = int(filename.split('_')[2])
    image_num = int(filename.split('_')[3].split('.')[0])
    dfs_o[(shreya_core_num,image_num)]=df_o

    
HI=6
MAX_HI = 50

def geno_hi(geno,df):
    return (df[geno]>=HI)&(df[geno]<=MAX_HI)
def geno_lo(geno,df):
    return (df[geno]>=1)&(df[geno]<HI)

for df_fish in list(dfs_o.values()):
    df_fish['EGFR_amp']=1*(geno_hi('EGFR',df_fish))
    df_fish['PDGFRA_amp']=1*(geno_hi('PDGFRA',df_fish))
    df_fish['CDK4_amp']=1*(geno_hi('CDK4',df_fish))
    
dfs={}    
for key,df in dfs_o.items():
        
    if key[0] not in tumor_core_to_shreya.values():
        continue

    dfs[key]=df[(df['EGFR']<=MAX_HI)&(df['PDGFRA']<=MAX_HI)&(df['CDK4']<=MAX_HI)]
 

In [ ]:
shreya_to_tumor={v:k[0] for k,v in tumor_core_to_shreya.items()}
core_list=list(set([item[0] for item in dfs.keys()]))
tumor_list = list(set([shreya_to_tumor[core] for core in core_list]))

tumor_image_dict={}
for k in dfs.keys():
    this_tumor=shreya_to_tumor[k[0]]
    if this_tumor in tumor_image_dict.keys():
        tumor_image_dict[this_tumor]=tumor_image_dict[this_tumor]+[k]
    else:
        tumor_image_dict[this_tumor]=[k]
        
dfs_in_tumor={}
for tumor, images in tumor_image_dict.items():
    dfs_in_tumor[tumor]=pd.concat([dfs[image] for image in images])

In [ ]:
tumor_stats_dict={}
for tumor in tumor_list:
    this_df=dfs_in_tumor[tumor]
    N_EC=len(this_df[(this_df['CDK4_amp']==1)&(this_df['EGFR_amp']==1)])
    N_C=len(this_df[(this_df['CDK4_amp']==1)&(this_df['EGFR_amp']==0)])
    N_E=len(this_df[(this_df['CDK4_amp']==0)&(this_df['EGFR_amp']==1)])
    N_NA=len(this_df[(this_df['CDK4_amp']==0)&(this_df['EGFR_amp']==0)])
    tumor_stats_dict[tumor]=(N_EC/N_E)/(N_C/N_NA)

In [ ]:
tumor_OR_df=pd.DataFrame.from_dict(tumor_stats_dict,orient='index').rename(columns={0:'OR'})

In [ ]:
q1=tumor_OR_df['OR'].quantile(1/3)
q2=tumor_OR_df['OR'].quantile(2/3)

In [ ]:
tumor_OR_df['Hi_OR']=1*(tumor_OR_df['OR']>q2)
tumor_OR_df['Lo_OR']=1*(tumor_OR_df['OR']<=q1)
tumor_OR_df['Med_OR']=1*((tumor_OR_df['OR']>q1)&(tumor_OR_df['OR']<=q2))

In [ ]:
tumor_OR_df['Hi_OR']='OR, top 1/3'
tumor_OR_df['Lo_OR']='OR, bottom 1/3'
tumor_OR_df['Med_OR']='OR, middle 1/3'

In [ ]:
tumor_OR_df['OR_char']=tumor_OR_df['Hi_OR']*(tumor_OR_df['OR']>q2)+tumor_OR_df['Lo_OR']*(tumor_OR_df['OR']<=q1)+tumor_OR_df['Med_OR']*((tumor_OR_df['OR']>q1)&(tumor_OR_df['OR']<=q2))

In [ ]:
c_neg='lightsteelblue'
c_neut='lightgray'
c_pos='salmon'
df_for_barchart=tumor_OR_df[['OR','OR_char']]
df_for_barchart['OR_char']=df_for_barchart['OR_char'].replace('OR, bottom 1/3',c_neg)
df_for_barchart['OR_char']=df_for_barchart['OR_char'].replace('OR, top 1/3',c_pos)
df_for_barchart['OR_char']=df_for_barchart['OR_char'].replace('OR, middle 1/3',c_neut)

In [ ]:
df_for_barchart=df_for_barchart.sort_values(by='OR')

In [ ]:
plt.rcParams["font.family"] = "Arial"
plt.rcParams["font.size"] = 14
fig, ax = plt.subplots()
lbls = list(df_for_barchart.index)
lbls=[str(lbl) for lbl in lbls]
ORs = list(df_for_barchart['OR'])

bar_colors=list(df_for_barchart['OR_char'])
plt.bar(lbls, ORs,color=bar_colors)
plt.axhline(1,color='darkgray')
  
plt.yscale('log')
plt.yticks([0.1,0.5,1,5,10])
ax.yaxis.set_major_formatter(matplotlib.ticker.ScalarFormatter())
x = np.arange(len(lbls))
ax.set_xticks(x)
ax.set_xticklabels(lbls)

plt.title('Odds Ratio for CDK4 amplification\nin the same cell as EGFR amplification',fontsize=14)
plt.ylabel('OR')
plt.xlabel('Tumor')
plt.tight_layout()
#plt.savefig('Odds_ratios_across_tumors_bar_plot.pdf')
plt.show()

In [ ]:
tumor_OR_df[['OR','OR_char']].to_csv('tumor_OR_df.csv')